## 机器学习评价指标汇总
### 分类
#### 精确率与召回率

In [1]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
accuracy_score(y_true, y_pred)

accuracy_score(y_true, y_pred, normalize=False)

#### ROC(Receiver operating characteristic)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
from sklearn.metrics import auc  ###计算roc和auc
import pandas as pd
 
base = "D:\\WFLW\\wflw_blur_128\\ROC\\"
 
df =pd.read_csv( base + "sobel_roc.txt", encoding='utf8', delimiter=' ', header=0)
df_lap =pd.read_csv( base + "lap_roc.txt", encoding='utf8', delimiter=' ', header=0)
df_dft =pd.read_csv( base + "dft_roc.txt", encoding='utf8', delimiter=' ', header=0)
df_cnn =pd.read_csv( base + "cnn_roc.txt", encoding='utf8', delimiter=' ', header=0)
 
tpr = df['tpr']
fpr = df['fpr']
 
tpr_lap = df_lap['tpr']
fpr_lap = df_lap['fpr']
 
tpr_dft = df_dft['tpr']
fpr_dft = df_dft['fpr']
 
tpr_cnn = df_cnn['tpr']
fpr_cnn = df_cnn['fpr']
 
roc_auc = auc(fpr, tpr) ###计算auc的值
 
roc_auc_lap = auc(fpr_lap, tpr_lap) ###计算auc的值
 
roc_auc_dft = auc(fpr_dft, tpr_dft) ###计算auc的值
 
roc_auc_cnn = auc(fpr_cnn, tpr_cnn)
 
print(roc_auc)
 
plt.figure()
lw = 2
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Sobel ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
 
plt.plot(fpr_lap, tpr_lap, color='navy', lw=lw, label='Lap ROC curve (area = %0.2f)' % roc_auc_lap)
 
plt.plot(fpr_dft, tpr_dft, color='red', lw=lw, label='Dft ROC curve (area = %0.2f)' % roc_auc_dft)
 
plt.plot(fpr_cnn, tpr_cnn, color='green', lw=lw, label='CNN ROC curve (area = %0.2f)' % roc_auc_cnn)
#plt.plot(tpr, fpr, 'k--', label='Mean ROC (area = %0.2f)' % 1, lw=2)
 
#plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
 
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.savefig(base + "roc_img\\sobel_roc.jpg")
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve
y = np.array([1, 1, 2, 2])
scores = np.array([0.1, 0.4, 0.35, 0.8])
fpr, tpr, thresholds = roc_curve(y, scores, pos_label=2)

#### 对数损失 Log loss

In [ ]:
import numpy as np
from sklearn.metrics import log_loss
value=log_loss(["spam", "ham", "ham", "spam"],[[.1, .9], [.9, .1], [.8, .2], [.35, .65]])
print("value:",value)


def logloss(true_label, predicted_prob):
  if true_label == 1:
    return -np.log(predicted_prob)
  else:
    return -np.log(1 - predicted_prob)

In [ ]:
from sklearn.metrics import log_loss
y_true = [0, 0, 1, 1]
y_pred = [[.9, .1], [.8, .2], [.3, .7], [.01, .99]]
log_loss(y_true, y_pred)

#### 铰链损失 Hinge loss

In [ ]:
from sklearn import svm
from sklearn.metrics import hinge_loss
X = [[0], [1]]
y = [-1, 1]
est = svm.LinearSVC(random_state=0)
est.fit(X, y)

pred_decision = est.decision_function([[-2], [3], [0.5]])
pred_decision

hinge_loss([-1, 1, 1], pred_decision)

#### 混淆矩阵 Confusion Matrix

In [ ]:
import numpy as np


def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


In [ ]:
from sklearn.metrics import confusion_matrix
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(y_true, y_pred)

#### kappa 系数

In [ ]:
def kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the kappa
    kappa calculates the kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            if i == j:
                d = 0.0
            else:
                d = 1.0
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator

In [ ]:
from sklearn.metrics import cohen_kappa_score
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
cohen_kappa_score(y_true, y_pred)

#### 准确率 Accuracy

In [ ]:
def get_classification_accuracy(feature_matrix, sentiment, coefficients):
    scores = np.dot(feature_matrix, coefficients)
    apply_threshold = np.vectorize(lambda x: 1. if x > 0  else -1.)
    predictions = apply_threshold(scores)
    
    num_correct = (predictions == sentiment).sum()
    accuracy = num_correct / len(feature_matrix)    
    return accuracy

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
scoring = {'accuracy': make_scorer(accuracy_score),
           'prec': 'precision'}

#### 海明距离 Hamming Distance

In [ ]:
import numpy as np
def hamming_distance(v1,v2)
smstr=np.nonzero(v1-v2)
sm= np.shape(smstr[0])[0]
return sm,smstr

In [ ]:
from sklearn.metrics import hamming_loss

y_pred =np.array([[0,1,1,1,0],
          [1,0,0,1,1],
          [1,1,0,0,0],
          [1,0,1,0,1]])

y_true =np.array([[1,1,1,0,0],
          [1,0,0,1,1],
          [1,0,1,0,1],
          [1,1,0,1,1]])

h=hamming_loss(y_true, y_pred)  
print("汉明损失：",h)

#### 杰卡德相似系数 Jaccard similarity coefficient

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score
 y_true = np.array([[0, 1, 1],
...                    [1, 1, 0]])
y_pred = np.array([[1, 1, 1],
...                    [1, 0, 0]])
jaccard_score(y_true[0], y_pred[0])

#### 多标签排序

##### 涵盖误差 Coverage error

In [ ]:
from sklearn.metrics import coverage_error
c=coverage_error(y_true, y_pred)-1
print("覆盖误差：",c)

##### 标签排序平均精度 Label ranking average precision

In [ ]:
from sklearn.metrics import average_precision_score
a=average_precision_score(y_true, y_pred) 
print("平均精度损失：",a)

### 回归
#### 平均绝对误差 MAE

In [ ]:
import numpy as np
# 平均绝对误差
def MAELoss(x:list,y:list):
    """
    x:list，代表模型预测的一组数据
    y:list，代表真实样本对应的一组数据
    """
    assert len(x)==len(y)
    x=np.array(x)
    y=np.array(y)
    loss=np.sqrt(np.sum(np.square(x - y)) / len(x))/n
    return loss

In [ ]:
from sklearn.metrics import mean_absolute_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_absolute_error(y_true, y_pred)

#### 平均平方误差 MSE

In [ ]:
import numpy as np
def MSELoss(x:list,y:list): #MSE
    """
    x:list，代表模型预测的一组数据
    y:list，代表真实样本对应的一组数据
    """
    assert len(x)==len(y)
    x=np.array(x)
    y=np.array(y)
    loss=np.sum(np.square(x - y)) / len(x)
    return loss

In [ ]:
from sklearn.metrics import mean_squared_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_squared_error(y_true, y_pred)
>>> 0.375

#### 解释变异系数

In [ ]:
from sklearn.metrics import explained_variance_score
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
explained_variance_score(y_true, y_pred)

#### 决定系数 $R^2$

In [ ]:
def R2Loss(x:list,y:list):
    """
    x:list，代表模型预测的一组数据
    y:list，代表真实样本对应的一组数据
    """
    assert len(x)==len(y)
    x=np.array(x)
    y=np.array(y)
    t=np.sum(y)/len(y)
    loss=1-((np.sum(np.square(x - y)) )/(np.sum(np.square(y-t))))
    loss=np.sqrt(np.sum(np.square(x - y)) / len(x))
    return loss

### 聚类
#### 兰德指数 Rand index

In [ ]:
def randIndex(clusters):

    answers = np.load('answersMatrixNOVO.npy')
    a = 0
    b = 0
    c = 0
    d = 0
    
    N = len(clusters)
    
    count = 0
    for i in range(0,N):
        for j in range(0,N):
            
            if answers[i,j] != 0.0:
                
                count = count + 1
            
                if clusters[i] == clusters[j]:
                    
                    if answers[i,j] == 2:
                        a = a + 1
                    else:
                        c = c + 1
                    
                else:
                    
                    if answers[i,j] == 1:
                        b = b + 1
                    else:
                        d = d + 1
                       
    rand_index = float(a + b)/(a + b + c + d)
    n = a+b+c+d
    aux = float((a+c)*(a+d) + (c+b)*(d+b))
    
    adjusted_rand_index = float(n*(a+b) - aux)/(pow(n,2) - aux)
    
    print 'a =' + str(a)
    print 'b =' + str(b)
    print 'c =' + str(c)
    print 'd =' + str(d)
    print 'Rand Index = ' + str(rand_index)
    print 'Adjusted Rand Index = ' + str(adjusted_rand_index)
    print 'Positive-negative coeficient index =' + str((float(a)/4635 + float(b)/15263)/2)
    
    return adjusted_rand_index

In [ ]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]
metrics.rand_score(labels_true, labels_pred)

#### 互信息 Mutual Information

In [ ]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]

metrics.adjusted_mutual_info_score(labels_true, labels_pred) 

#### 轮廓系数 Silhouette coefficient

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_
metrics.silhouette_score(X, labels, metric='euclidean')